In [1]:
# Install/load packages

paket <- function(pak){
  not_installed <- pak[!(pak %in% rownames(installed.packages()))]
  if (length(not_installed))
    install.packages(not_installed, dependencies = TRUE,repos='http://cran.us.r-project.org')
  sapply(pak, library, character.only = TRUE)
}

install = c("shiny", "shinydashboard", "shinyWidgets", "esquisse", "dplyr", "DT", "stringr", "tidyr")
paket(install)


Attaching package: ‘shinydashboard’


The following object is masked from ‘package:graphics’:

    box



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


This version of Shiny is designed to work with 'htmlwidgets' >= 1.5.
    Please upgrade via install.packages('htmlwidgets').


Attaching package: ‘DT’


The following objects are masked from ‘package:shiny’:

    dataTableOutput, renderDataTable




$shiny
[1] "shiny"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

$shinydashboard
[1] "shinydashboard" "shiny"          "stats"          "graphics"      
[5] "grDevices"      "utils"          "datasets"       "methods"       
[9] "base"          

$shinyWidgets
 [1] "shinyWidgets"   "shinydashboard" "shiny"          "stats"         
 [5] "graphics"       "grDevices"      "utils"          "datasets"      
 [9] "methods"        "base"          

$esquisse
 [1] "esquisse"       "shinyWidgets"   "shinydashboard" "shiny"         
 [5] "stats"          "graphics"       "grDevices"      "utils"         
 [9] "datasets"       "methods"        "base"          

$dplyr
 [1] "dplyr"          "esquisse"       "shinyWidgets"   "shinydashboard"
 [5] "shiny"          "stats"          "graphics"       "grDevices"     
 [9] "utils"          "datasets"       "methods"        "base"          

$DT
 [1] "DT"             "dplyr"          "esquisse"       "shinyWidgets"  
 [5] "shinydashboard" "shiny"          "stats"          "graphics"      
 [9] "grDevices"      "utils"          "datasets"       "methods"       
[13] "base"          

$stringr
 [1] "stringr"        "DT"             "dplyr"          "esquisse"      
 [5] "shinyWidgets"   "shinydashboard" "shiny"          "stats"         
 [9] "graphics"       "grDevices"      "utils"          "datasets"      
[13] "methods"        "base"          

$tidyr
 [1] "tidyr"          "stringr"        "DT"             "dplyr"         
 [5] "esquisse"       "shinyWidgets"   "shinydashboard" "shiny"         
 [9] "stats"          "graphics"       "grDevices"      "utils"         
[13] "datasets"       "methods"        "base"

# Installing PICSURE

In [2]:
source("requirements.R")

installing: 
-  ggplot2 
-  dplyr 
-  tidyr 
-  urltools 
-  devtools 
-  ggrepel 


Loading required package: usethis



In [3]:
devtools::install_github("hms-dbmi/pic-sure-r-client", force = T, upgrade = "always")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force = T, upgrade = "always")


✔  checking for file ‘/private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/RtmpwerjcF/remotes1116400ae46d/hms-dbmi-pic-sure-r-client-902762b/DESCRIPTION’ (441ms)
─  preparing ‘picsure’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘picsure_0.1.0.tar.gz’
   



✔  checking for file ‘/private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/RtmpwerjcF/remotes11161b8715fe/hms-dbmi-pic-sure-r-adapter-hpds-c582192/DESCRIPTION’
─  preparing ‘hpds’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘hpds_0.1.0.tar.gz’
   


# Loading data

In [4]:
source("R_lib/wrappers.R")

In [16]:
whole_data <- data.table::fread(file = "multiIndex_variablesDict.csv",
                                sep=",",
                                header = TRUE,
                                na.strings = c("NA", "")) 

# Data Management

In [18]:
whole_data = whole_data %>% select("simplified_name", "name", everything())
study_names <- whole_data$level_0 %>% unique()
not_all_na <- function(x) {!all(is.na(x))}
less_20_elem <- function(x) {length(unique(x)) < 20}
whole_data[["names_wout_backslashes"]] <- gsub("\\", " ", whole_data[["name"]], fixed=TRUE)

In [19]:
## REFERENCES ##
# https://rstudio.github.io/shinydashboard/structure.html
# https://rstudio.github.io/shinydashboard/appearance.html


app <- shinyApp(

 
   {
      #### UI HEADER ####
      DBheader <- dashboardHeader(title="BDCatalyst Variable Browser",
                                  titleWidth = 350 )


      DBsidebar <- dashboardSidebar(width = 350,
                                    fluidRow(column(8,
                                             searchInput(
                                          inputId="variable_search_box",
                                          label = "Search Variable Names",
                                          value = "",
                                          placeholder = "eg: Smoking",
                                          btnSearch = NULL,
                                          btnReset = NULL,
                                          resetValue = "",
                                          width = NULL
                                       )),column(4,
                                    checkboxInput(
                                          "search_regex", "regex", value = FALSE, width = NULL
                                    ))),
                                    filterDF_UI("filtering")
      )



      #### UI BODY ####
      DBbody <- dashboardBody(
         fluidRow(
            pickerInput(
               inputId = "subset",
               label = "Select specific BDC Study:",
               choices = study_names,
               selected = study_names,
               multiple = T,
               options = list('actions-box' = T),
               inline = F,
            )
         ),
         fluidRow(
            column(width = 12,
                   box(
                      width = NULL,
                      div(style = 'overflow-x: scroll',
                          DT::dataTableOutput(outputId = "table")),
                      progressBar(
                         id = "pbar", value = 100,
                         total = 100, display_pct = TRUE
                      ),
                      # tags$p("Code dplyr:"),
                      # verbatimTextOutput(outputId = "code_dplyr"),
                      # tags$p("Expression:"),
                      # verbatimTextOutput(outputId = "code"),
                      # tags$p("Filtered data:"),
                      # verbatimTextOutput(outputId = "res_str")
                   )
            )
         )
      )



      #### GENERATING UI ####
      ui <- dashboardPage(
         skin="yellow",
         DBheader,
         DBsidebar,
         DBbody
      )

   },

   server <- function(input, output, session) {

      data <- reactive({
         test = whole_data %>%
            filter(level_0 %in% input$subset) %>%
            select_if(not_all_na)
         if ((input$variable_search_box != "") & isTRUE(input$search_regex)) {
            filter(test, stringr::str_detect(test$names_wout_backslashes, input$variable_search_box))
         } else if ((input$variable_search_box != "") & isFALSE(input$search_regex)) {
            filter(test, stringr::str_detect(test$names_wout_backslashes, stringr::coll(input$variable_search_box, TRUE)))
         } else {
            test
         }
      })

            res_filter <- callModule(
               module = filterDF,
               id = "filtering",
               data_table = data,
               data_name = reactive("Filtered Variable names")
            )

            observeEvent(res_filter$data_filtered(), {
               updateProgressBar(
                  session = session, id = "pbar",
                  value = nrow(res_filter$data_filtered()), total = nrow(data())
               )
            })

            output$table <- DT::renderDT({
               res_filter$data_filtered() %>%
                  DT::datatable(.,
                                caption = "You can rearrange columns order by drag and drop",
                                escape = FALSE, filter = 'top', rownames = FALSE,
                                extensions = list('ColReorder' = NULL, 'RowReorder' = NULL,
                                                  'Buttons' = NULL),
                                options = list(dom = 'BRrltpi',
                                               lengthMenu = list(c(10, 50, 100, -1), c('10', '50', '100', 'All')),
                                               pageLength = 50,
                                               ColReorder = TRUE,
                                               rowReorder = TRUE,
                                               buttons = list(I('colvis'), list(
                                                  extend = "collection",
                                                  buttons = c('copy', 'csv', 'pdf'),
                                                  text = "Export"
                                               )),
                                               columnDefs = list(list(visible=FALSE, targets = which(names(.) == "names_wout_backslashes") - 1)
                                                                 )
                                )
                  ) %>% DT::formatStyle(
                     c("simplified_name", "name"),
                     backgroundColor = 'lightgreen')

            }, options = list(pageLength = 100))


            output$code_dplyr <- renderPrint({
               res_filter$code$dplyr
            })
            output$code <- renderPrint({
               res_filter$code$expr
            })

            output$res_str <- renderPrint({
               str(res_filter$data_filtered())
            })

   }
)

In [ ]:
browseURL("http://127.0.0.1:4868")
options(shiny.port = 4868)
runApp(app, host = getOption("shiny.host", "127.0.0.1"))


Listening on http://127.0.0.1:4868

Warning message in processWidget(instance):
“renderDataTable ignores ... arguments when expr yields a datatable object; see ?renderDataTable”
Warning message in processWidget(instance):
“renderDataTable ignores ... arguments when expr yields a datatable object; see ?renderDataTable”
